# 2021.06.19 제 2회 빅분기 복기

## 작업형 1유형

### 문제1
- 주어진 데이터셋에서 'CRIM'값이 가장 큰 10개지역을 구하고
- 10개 지역의 'CRIM' 값을 그중 가장 작은 값으로 대체 
- 'AGE' 컬럼값이 80이상인 행의 'CRIM'평균값을 구해라(단 출력 결과는 정수 형태로 한다.)

In [23]:
import pandas as pd

data = pd.read_csv('https://raw.githubusercontent.com/JEunJin/Bigdata_python/master/bigdata_csvfile/boston_housing_data.csv')

data = data.sort_values('CRIM',ascending=False) # 내림차순 

#가장 큰 10개 지역 
#print(data.iloc[0:10]) # 이중 가장 작은 값 = 25.9406
min_crim = data.iloc[9]['CRIM']
#print(min_crim) # 
data.iloc[0:10,0] = min_crim # CRIM의 index = 0
#print(data.iloc[0:10]) # 이중 가장 작은 값 으로 대체 완료 

data = data[data['AGE'] >=80]# AGE 80 이상 인 것들만 가져옴옴
print(int(data['CRIM'].mean()))



5


### 문제 2
- 첫번째 행부터 순서대로 80%까지 데이터 추출
- total bedrooms 컬럼의 중앙값으로 해당 컬럼의 결측치를 대체 
- total bedrooms컬럼의 대치 전후의 표준편차 차이 구헤라(단 출력값은 소수점 3자리까지 출력력)

In [29]:
import pandas as pd 
data = pd.read_csv('https://raw.githubusercontent.com/JEunJin/Bigdata_python/master/bigdata_csvfile/california_housing.csv')
#print(data.info()) # total bedrooms 에 결측치 존재 
#print(20640*0.8) # 16512 가 전체 80

data = data.iloc[0:16512]
before_std = data['total_bedrooms'].std()
#print(data.info())
data['total_bedrooms'].fillna(data['total_bedrooms'].median(),inplace=True)
#print(data.info()) 결측치 대체 완료 
after_std = data['total_bedrooms'].std()

print(round(before_std-after_std,3))



1.975


C:\Users\yee77\AppData\Local\Temp\ipykernel_17892\252969956.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['total_bedrooms'].fillna(data['total_bedrooms'].median(),inplace=True)


### 문제 3
- population 컬럼의 평균으로부터 1.5x표준편차를 벗어나는 영역을 이상치로 판단
- 이상치들의 합을 구하시오 


In [35]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/JEunJin/Bigdata_python/master/bigdata_csvfile/california_housing.csv')

m = data['population'].mean()
s = data['population'].std()
#print(m,s)
lower = m - 1.5*s
upper = m + 1.5*s
#print(upper,lower)
data = data[(data['population']>upper) | (data['population']<lower)]
print(data['population'].sum())

5607295.0


## 작업형 2유형 

### 문제 
- 주어진 데이터 활용 , 고객이 주문한 물품의 제시간 도착 여부를 예측하는 모델 생성(Reached.on_Time_Y.N)
- 평가기준은 roc_auc_score
- 제출형태: 'cust_id':ID , 'reached_ontime':value 
- 제출파일 이름 : 2nd_test_type2.csv 
- 데이터 상위 10000개 데이터를 train으로 사용 , 나머지 999개를 test로 사용 할 수 있도록 슬라이싱


In [57]:
import pandas as pd 
data = pd.read_csv('https://raw.githubusercontent.com/JEunJin/Bigdata_python/master/bigdata_csvfile/E-Commerce_Shipping.csv')
#print(data.info()) # 결측치 없음 
# 전처리(4개 컬럼)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['Warehouse_block'] = le.fit_transform(data['Warehouse_block'])
data['Mode_of_Shipment'] = le.fit_transform(data['Mode_of_Shipment'])
data['Product_importance'] = le.fit_transform(data['Product_importance'])
data['Gender'] = le.fit_transform(data['Gender'])

# 데이터 슬라이싱 
train = data.iloc[0:10000]
test = data.iloc[10000:]
#print(train.info())

# 데이터 분할
X = train.drop(columns=['Reached.on.Time_Y.N'])
y = train['Reached.on.Time_Y.N']
from sklearn.model_selection import train_test_split
train_X,val_X,train_y,val_y=train_test_split(X,y,test_size=0.2,random_state=123)

# 분류 모델 학습 
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(train_X,train_y)
y_pred = rfc.predict(val_X)
# 평가 
from sklearn.metrics import roc_auc_score,accuracy_score
print(roc_auc_score(y_pred,val_y)) # 0.68
print(accuracy_score(y_pred,val_y)) # 0.68  >> 이정도면 무난한 모델?

# 최종 결과 예측 
test_X = test.drop(columns=['Reached.on.Time_Y.N'])
test_y = test['Reached.on.Time_Y.N']
pred_y = rfc.predict(test_X)
#print(roc_auc_score(pred_y,test_y)) # 0.49
#print(accuracy_score(pred_y,test_y)) # 0.54  >> 잘 안나오는데?
# 최종결과 제출

pd.DataFrame({'cust_id':test['ID'], 'reached_ontime':pred_y}).to_csv('2nd_test_type2.csv',index=False)
print(pd.read_csv('2nd_test_type2.csv'))


0.6812909793359945
0.686
     cust_id  reached_ontime
0      10001               1
1      10002               0
2      10003               1
3      10004               1
4      10005               0
..       ...             ...
994    10995               0
995    10996               0
996    10997               0
997    10998               0
998    10999               0

[999 rows x 2 columns]
